In [1]:
def result(G, output, label_idx, idx2label):
    SF = nn.Softmax(dim=-1)
    score, pred = SF(output).max(dim=-1)
    sms = [G.node[i]['text'] for i in label_idx]
    df = pd.DataFrame()
    df['sms']= sms
    df['score']=score.data
    df['pred']=[idx2label[i] for i in pred.numpy()]
    df['label']=[g4.node[i]['label'] for i in label_idx]
    return df

def get_clf_prediction(model, X_word, X_char, y):
    print('X shape:{}, y_shape:{}'.format(len(X_word), len(y)))
    y_pred = model.predict([X_word,
                            np.array(X_char).reshape((len(X_char),
                                                         max_len, max_len_char))])
    pred = [idx2label[i] for i in np.argmax(y_pred,axis=-1)]
    true = [idx2label[i] for i in y]
    return true, pred

def get_pr(Res, cls):
    #某类 正确识别数量/该类 总识别数量
    tmp  = Res[Res.true == cls]
    a = sum(tmp.true.values == tmp.pred.values)
    b = Res[Res.pred == cls].shape[0]
    pr = a/b
    return pr

def get_rc(Res, cls):
    #某类 正确识别数量/该类 总数量
    tmp  = Res[Res.true == cls]
    support = tmp.shape[0]
    a = sum(tmp.true.values == tmp.pred.values)
    b = Res[Res.true == cls].shape[0]
    rc = a/b
    return rc,support

def get_f1(pr, rc):
    # f1 = (2*pr*rc)/(pr+rc)
    f1 = (2*pr*rc)/(pr+rc)
    return f1

def evaluate(Res, cls):
    pr = get_pr(Res, cls)
    rc, support = get_rc(Res, cls)
    f1 = get_f1(pr, rc)
    return [cls, pr, rc, f1, support]

def Final_evalu(z):
    Res = pd.DataFrame(columns=['true', 'pred'])
    true, pred = z.label.tolist(), z.pred.tolist()
    Res['true'] = true
    Res['pred'] = pred
    entitys = Res.true.unique()
    records = []
    for i in entitys:
        tmp = evaluate(Res, i)
        records.append(tmp)
    record = pd.DataFrame.from_records(records)
    record.columns =['cls','精确率','召回率', 'F1', 'support']
    record = record.set_index('cls')
    record = record.sort_index()   
    return record


In [2]:
import networkx as nx
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from numpy import dot
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
import torch
import torch.nn as nn
import copy
# from graphviz import Digraph
from torch.nn import init
from torch.autograd import Variable
import torch.nn.functional as F
import time
import random
from sklearn.metrics import f1_score
# from collections import defaultdict
# from ClfTemplates import get_all_template_path, sample_template, sample_template_path
# from ClfUse import showClfResult, get_samples, labeledDataFromJson
# from Ner_model import get_X_Y, lstm_crf, predict, preprocess
# from Ner_data_make import create_fake_label, fake2real
from keras.preprocessing.sequence import pad_sequences

from keras.models import Model, Input, Sequential
from keras.layers import LSTM, Embedding, SpatialDropout1D,concatenate, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras.utils import to_categorical
import keras
from collections import Iterable 
import pandas as pd
import numpy as np
import random
import re
import warnings
warnings.filterwarnings("ignore")


%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [3]:
def get_cos_similarity(sms, templates):
    '''计算一条新sms 与 每个 template 相似度;'''
    def cos_sim(a, b):
        return dot(a, b) / (norm(a) * norm(b))
    return [cos_sim(i, sms) for i in templates]


def tfIdfVector(corpus):
    '''
    corpus is a list of sentences:
    ['This is an example', 'hello world', ...]
    '''
    vectorizer = CountVectorizer()
    transformer = TfidfTransformer()
    x = vectorizer.fit_transform(corpus)
    tfidf = transformer.fit_transform(x)
    return tfidf.toarray()


In [4]:
"""
Set of modules for aggregating embeddings of neighbors.
"""
class MeanAggregator_QBv2(nn.Module):
    """
    adjMat@embMat: [bs, uniqueNode]@[uniqueNode, F]=[bs, F]
    
    要改，之前没注意每次采样全图遍历；
    如无必要，不要让一个实例挂一个毒瘤: G (20G内存)
    """
    def __init__(self, features, cuda=False, gcn=False): 
        """
        Initializes the aggregator for a specific graph.
        features -- function mapping LongTensor of node ids to FloatTensor of feature values.
        cuda -- whether to use GPU;
        gcn --- whether to perform concatenation GraphSAGE-style, or add self-loops GCN-style
        """
        super(MeanAggregator_QBv2, self).__init__()
        self.features = features
        self.cuda = cuda
        self.gcn = gcn
        
    def forward(self, nodes, to_neighs, num_sample=10):
        """
        nodes --- list of nodes in a batch, [bs]
        to_neighs --- list of sets, 
                      each set is the set of neighbors for node in batch
        num_sample --- number of neighbors to sample. 
        """
        _set = set
        if not num_sample is None:
            _sample = random.sample
            samp_neighs = [_set(_sample(to_neigh, num_sample))
                           if len(to_neigh) >= num_sample else to_neigh 
                           for to_neigh in to_neighs]
        else:
            samp_neighs = to_neighs
    
        # 纳入target idx;
        if self.gcn:
            samp_neighs = [set.union(set(samp_neigh), set([nodes[i]])) for i, samp_neigh in enumerate(samp_neighs)]
        unique_nodes_list = list(set.union(*samp_neighs))
        unique_nodes = {n:i for i,n in enumerate(unique_nodes_list)}
        mask = Variable(torch.zeros(len(samp_neighs), len(unique_nodes)))
        column_indices = [unique_nodes[n] for samp_neigh in samp_neighs for n in samp_neigh]   
        row_indices = [i for i in range(len(samp_neighs)) for j in range(len(samp_neighs[i]))]
        mask[row_indices, column_indices] = 1
        if self.cuda:
            mask = mask.cuda()
        num_neigh = mask.sum(1, keepdim=True)
        mask = mask.div(num_neigh)
        if self.cuda:
            embed_matrix = self.features(torch.LongTensor(unique_nodes_list).cuda())
        else:
            embed_matrix = self.features(torch.LongTensor(unique_nodes_list))
        to_feats = mask.mm(embed_matrix)
        return to_feats

    
class Encoder_QBv2(nn.Module):
    """
    ([cls, 2f]@[2f, bs]).T = [bs, cls] 
    Encodes a node's using 'convolutional' GraphSage approach
    """
    def __init__(self, features, feature_dim, 
            embed_dim, adj_lists, aggregator,
            num_sample=10,
            base_model=None, gcn=False, cuda=False, 
            feature_transform=False): 
        super(Encoder_QBv2, self).__init__()

        self.features = features
        self.feat_dim = feature_dim
        self.adj_lists = adj_lists
        self.aggregator = aggregator
        self.num_sample = num_sample
        if base_model != None:
            self.base_model = base_model

        self.gcn = gcn
        self.embed_dim = embed_dim
        self.cuda = cuda
        self.aggregator.cuda = cuda
        self.weight = nn.Parameter(
                torch.FloatTensor(embed_dim, self.feat_dim if self.gcn else 2 * self.feat_dim))
        init.xavier_uniform(self.weight)


    def forward(self, nodes):
        """
        Generates embeddings for a batch of nodes.
        nodes     -- list of nodes; [bs]
        operations: 已知self_feat: [bs, F]
                    再agg_feat: [bs, F]
                    然后concat([agg_feat, self_feat]): [bs, 2F]
                    然后W:[F2, 2F] 
                    relu(W @[bs,2F].T) : [F2, bs] 
        """
        neigh_feats = self.aggregator.forward(nodes, [self.adj_lists[int(node)] for node in nodes], 
                self.num_sample)
        
        if not self.gcn:
            if self.cuda:
                self_feats = self.features(torch.LongTensor(nodes).cuda())
            else:
                self_feats = self.features(torch.LongTensor(nodes))
            combined = torch.cat([self_feats, neigh_feats], dim=1)
        else:
            combined = neigh_feats
        combined = F.relu(self.weight.mm(combined.t()))
        return combined


class SupervisedGraphSage_QB_v3(nn.Module):
    def __init__(self, num_classes, enc, features):
        super(SupervisedGraphSage_QB_v3, self).__init__()
        self.enc = enc
        self.features = features
        self.weight = nn.Parameter(torch.FloatTensor(num_classes, enc.embed_dim))
        init.xavier_uniform(self.weight)
        self.loss = nn.NLLLoss(reduce=False)
        self.logsoftmax = nn.LogSoftmax(dim=-1)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, nodes, softmax=False):
        embeds = self.enc(nodes)
        scores = self.weight.mm(embeds)
        if softmax:
            return self.softmax(scores.t())
        return scores.t()
            

    def CompMaskLoss(self, nodes, neighs, negNeighs, labels, mask=None):
        ''' comp loss including both clf & nce loss;
        :nodes: batch nodes id with shape :[bs];
        :neighs: positive neighbours of nodes with shape: [bs, K], K is num of neighs for each node;
        :negNeighs: negNeighs of nodes with shape: [bs, K];
        :labels: batch labels with shape:[bs]
        :mask: only part of samples will go backwards prop; shape: [bs];
        '''
        if mask is None:
            mask = torch.ones(len(labels))
        logits = self.forward(nodes)
        A_soft = self.logsoftmax(logits)
        
        output = self.loss(A_soft, torch.tensor(labels.squeeze()))
        clf_loss = output*mask.type(torch.float)
        clf_loss = sum(clf_loss)/sum(mask)
        nce_loss = self._nec_forward(nodes, neighs, negNeighs)
        return clf_loss, nce_loss, clf_loss + nce_loss
    
    def _nec_forward(self, nodes, neighs, negNeighs):
        '''unsupervised nce loss;
        :nodes: barch node ids [bs];
        :neighs: positive samples with regard to nodes [bs, K], K is num of neighs for each node;
        :negNeighs: negative samples with regrad to nodes [bs, K];'''
        bs = len(nodes)
        #bs, F, 1
        nodes = self.features(torch.LongTensor(nodes)).unsqueeze(-1)
        #bs, 1, F
        pos_nodes = torch.stack([self.features(torch.LongTensor(neighs[i])) for i in range(len(neighs))])
        #bs, 5, F
        neg_nodes = torch.stack([self.features(torch.LongTensor(negNeighs[i])) for i in range(len(negNeighs))])
        
        sum_log_neg = torch.bmm(neg_nodes, nodes).neg().sigmoid().log().squeeze().sum()
        sum_log_pos = torch.bmm(pos_nodes, nodes).sigmoid().log().squeeze().sum() 
#         print('sum neg loss:{}/n sum pos loss:{}'.format(sum_log_neg, sum_log_pos))
        return (sum_log_pos + sum_log_neg)/bs

    
def GraphBuild_v2(template_df, K, train=True):
    '''get samples before feed into the NN;
    : template_df: a df with 'sms' and ['cls' cols(only for training mode)];
    : K: num of neighs that will be sampled for each target node;
    : train: training mode or prediction mode. default True;
    '''    
    def getHardSamples(adj_mat, labels, target_id, K, sample_negative = True):
        '''Give one node id, find its top K hard negative(or positive) node_ids;
           Only for training mode;
           note that the return list of lists may not have same length;
        :paras adj_mat: adj matrix;
        :labels: node labels, each label corresponding with adj_mat;
        :target_id: given one node id, and find its top K hard negative node ids;
        :K: only return top K hard negative samples;
        '''
        if sample_negative:
            sortNode = sorted(zip(adj_mat[target_id], labels, range(len(labels))), key=lambda x: x[0], reverse=True)
            targetLabel = labels[target_id]
            nodeId = [i for i in sortNode if i[1] != targetLabel][:K]
        else:
            sortNode = sorted(zip(adj_mat[target_id], labels, range(len(labels))), key=lambda x: x[0])
            targetLabel = labels[target_id]
            nodeId = [i for i in sortNode if i[1] == targetLabel][:K]
        return [i[2] for i in nodeId]
    
    def findNeighsK(arr, K):
        '''Only for prediction mode since <getHardSamples> function only works for training mode;'''
        K+=1 #入图时neighs 不包括自己，故这里先加一个保证去除自己之后仍然有K neighs;
        idx=np.argpartition(arr, -K)[-K:]
        return idx
    
    if train:
        labels = template_df.cls.tolist()
    corpus = template_df.sms.tolist()
    all_tfidf = tfIdfVector(corpus)
    adj_mat = cosine_similarity(all_tfidf)
    print('adj mat shape:{}'.format(adj_mat.shape))
    G = nx.Graph()
    G.add_nodes_from(range(len(adj_mat)))
    for i in range(len(G.nodes())):
        G.node[i]['vec'] =  all_tfidf[i]
        G.node[i]['text'] = corpus[i]
        if train:
            G.node[i]['label'] = labels[i]
            # 正例可能找不满K个；因为数据集太小了；因此k=1;
            G.node[i]['neighs_k'] = getHardSamples(adj_mat, labels, i, 1, False) 
            G.node[i]['NegNeighs_k'] = getHardSamples(adj_mat, labels, i, K, True) 
        else:
            G.node[i]['neighs_k'] = set(findNeighsK(adj_mat[i], K)) - set([i])
    return G

    
    
def result(G, output, label_idx, idx2label):
    score, pred = output.max(dim=-1)
    sms = [G.node[i]['text'] for i in label_idx]
    df = pd.DataFrame()
    df['sms']= sms
    df['score']=score.data
    df['pred']=[idx2label[i] for i in pred.numpy()]
    df['label']=[G.node[i]['label'] for i in label_idx]
    return df

def load_india_sms(G, train=True):
    if train:
        labels = [G.node[i]['label'] for i in G.nodes]
        NegNeighs = [G.node[i]['NegNeighs_k'] for i in G.nodes]
    else:
        labels = None
        NegNeighs = None
    feat_data = [G.node[i]['vec'] for i in G.nodes]
    PosNeighs = [G.node[i]['neighs_k'] for i in G.nodes]
    return feat_data, labels, PosNeighs, NegNeighs


In [5]:
gsm_templates_df = pd.read_csv('gsm_templates_df.csv')
gsm_templates_df.dropna(inplace=True)
gsm_templates_df.columns = ['sms', 'cls']
test_df = gsm_templates_df.sample(100)
train_df = gsm_templates_df[~gsm_templates_df.index.isin(test_df.index)]

g5 = GraphBuild_v2(train_df, 5, True)
g5.node[0]


adj mat shape:(1095, 1095)


{'NegNeighs_k': [97, 1009, 12, 569, 1011],
 'label': '交易流水＿转账',
 'neighs_k': [379],
 'text': 'dear customer txn on indusind bank credit card no xx5007 for inr 2741 on 26/08/18 18:20 at rajpath motors is approved.pls call 18602677777 for query.click on http://bit.ly/2b8nsnl to update your aadhaar number, ignore if already done.',
 'vec': array([0., 0., 0., ..., 0., 0., 0.])}

In [6]:
np.random.seed(1)
random.seed(1)
feat_data, labels, PosNeighs, NegNeighs = load_india_sms(g5, train=True)
idx2label = {i:j for i, j in enumerate(np.unique(labels))}
label2idx = {j:i for i,j in idx2label.items()}
labels = np.array([label2idx[i] for i in labels])
num_cls = len(np.unique(labels))

features = nn.Embedding(len(feat_data), len(feat_data[0]))
features.weight = nn.Parameter(torch.FloatTensor(feat_data), requires_grad=True)
node_F = len(feat_data[0])
agg1 = MeanAggregator_QBv2(features, cuda=False, gcn=True)
enc1 = Encoder_QBv2(features, node_F, 128, PosNeighs, agg1, num_sample=5, gcn=True, cuda=False)
agg2 = MeanAggregator_QBv2(lambda nodes : enc1(nodes).t(), cuda=False, gcn=True)
enc2 = Encoder_QBv2(lambda nodes : enc1(nodes).t(), 
               enc1.embed_dim, node_F, PosNeighs, agg2, num_sample=5, gcn=True, cuda=False)

In [7]:
graphsage = SupervisedGraphSage_QB_v3(num_cls, enc2, features)
nodes = list(g5.nodes)
val = nodes[:50]
test_nodes = list(g5.nodes)[-test_df.shape[0]:]
train = nodes[50:-len(test_nodes)]
optimizer = torch.optim.SGD(filter(lambda p : p.requires_grad, graphsage.parameters()), lr = 0.1)
times = []
print('GO')

for batch in range(1400):
    batch_nodes2 = train[:200]
    batch_idx = copy.copy(train[:200])
    random.shuffle(train)
    start_time = time.time()
    optimizer.zero_grad()
    

    clf, nec, loss = graphsage.CompMaskLoss(batch_nodes2,
                                  np.array(PosNeighs)[[batch_idx]],
                                  np.array(NegNeighs)[[batch_idx]],
                                  np.array(labels)[[batch_idx]])
    loss.backward()
    optimizer.step()
    end_time = time.time()
    times.append(end_time - start_time)

    if batch%100 == 0:
        tra_output = graphsage.forward(train)
        train_res = result(g5, tra_output, train, idx2label)
        print(Final_evalu(train_res))
        
        val_output = graphsage.forward(val)
        val_res = result(g5, val_output, val, idx2label)
        print(Final_evalu(val_res))

        print('''batch train loss\n clf:{}\n nce:{}\n total:{}\n'''.format(clf.data, nec.data, loss.data))
        val_clf, val_nce, val_total = graphsage.CompMaskLoss(val,
                                  np.array(PosNeighs)[[val]],
                                  np.array(NegNeighs)[[val]],
                                  np.array(labels)[[val]])
        print('batch val loss\n clf:{}\n nce:{}\n total:{}\n'.format(val_clf.data, val_nce.data, val_total.data))
        print(' ###################### batch :{}#################################'.format(batch))
        

GO
                  精确率       召回率        F1  support
cls                                               
sms_other    0.000000  0.000000       NaN       35
交易流水＿余额      0.026144  0.150943  0.044568       53
交易流水＿转账      0.333333  0.001639  0.003263      610
信用卡＿申请失败          NaN  0.000000       NaN        2
信用卡＿还款提醒     0.046512  0.044444  0.045455       45
信用卡＿逾期警告          NaN  0.000000       NaN       13
账号异常＿余额不足    0.000000  0.000000       NaN       56
账号异常＿信用额度不足  0.000000  0.000000       NaN        1
账号异常＿卡号冻结         NaN  0.000000       NaN       19
账号异常＿扣款失败    0.000000  0.000000       NaN       55
贷前申请＿审核拒绝         NaN  0.000000       NaN        3
贷前申请＿审核通过         NaN  0.000000       NaN        9
贷前申请＿申请交互    0.000000  0.000000       NaN       10
贷后提醒＿到期提醒         NaN  0.000000       NaN       15
贷后提醒＿成功放款    0.000000  0.000000       NaN        3
贷后提醒＿逾期催收    0.000000  0.000000       NaN       16
                精确率  召回率        F1  support
cls                                

                  精确率  召回率        F1  support
cls                                          
sms_other         NaN  0.0       NaN       35
交易流水＿余额           NaN  0.0       NaN       53
交易流水＿转账      0.645503  1.0  0.784566      610
信用卡＿申请失败          NaN  0.0       NaN        2
信用卡＿还款提醒          NaN  0.0       NaN       45
信用卡＿逾期警告          NaN  0.0       NaN       13
账号异常＿余额不足         NaN  0.0       NaN       56
账号异常＿信用额度不足       NaN  0.0       NaN        1
账号异常＿卡号冻结         NaN  0.0       NaN       19
账号异常＿扣款失败         NaN  0.0       NaN       55
贷前申请＿审核拒绝         NaN  0.0       NaN        3
贷前申请＿审核通过         NaN  0.0       NaN        9
贷前申请＿申请交互         NaN  0.0       NaN       10
贷后提醒＿到期提醒         NaN  0.0       NaN       15
贷后提醒＿成功放款         NaN  0.0       NaN        3
贷后提醒＿逾期催收         NaN  0.0       NaN       16
            精确率  召回率        F1  support
cls                                    
sms_other   NaN  0.0       NaN        1
交易流水＿余额     NaN  0.0       NaN        2
交易流水＿转账    0

                  精确率       召回率        F1  support
cls                                               
sms_other    0.000000  0.000000       NaN       35
交易流水＿余额      0.955556  0.811321  0.877551       53
交易流水＿转账      0.706019  1.000000  0.827680      610
信用卡＿申请失败          NaN  0.000000       NaN        2
信用卡＿还款提醒          NaN  0.000000       NaN       45
信用卡＿逾期警告          NaN  0.000000       NaN       13
账号异常＿余额不足    0.848485  0.500000  0.629213       56
账号异常＿信用额度不足       NaN  0.000000       NaN        1
账号异常＿卡号冻结         NaN  0.000000       NaN       19
账号异常＿扣款失败    0.000000  0.000000       NaN       55
贷前申请＿审核拒绝         NaN  0.000000       NaN        3
贷前申请＿审核通过         NaN  0.000000       NaN        9
贷前申请＿申请交互         NaN  0.000000       NaN       10
贷后提醒＿到期提醒         NaN  0.000000       NaN       15
贷后提醒＿成功放款         NaN  0.000000       NaN        3
贷后提醒＿逾期催收         NaN  0.000000       NaN       16
                精确率   召回率     F1  support
cls                                     

In [8]:
te_output = graphsage.forward(test_nodes, True)
test_res = result(g5, te_output, test_nodes, idx2label)
Final_evalu(test_res)

,精确率,召回率,F1,support
cls,,,,
sms_other,NaN,0.0,NaN,4
交易流水＿余额,0.750000,1.0,0.857143,3
交易流水＿转账,0.700000,1.0,0.823529,56
信用卡＿还款提醒,NaN,0.0,NaN,19
信用卡＿逾期警告,NaN,0.0,NaN,5
账号异常＿余额不足,0.500000,1.0,0.666667,1
账号异常＿卡号冻结,NaN,0.0,NaN,1
账号异常＿扣款失败,0.285714,1.0,0.444444,4
贷前申请＿审核拒绝,NaN,0.0,NaN,1


In [9]:
tra_output = graphsage.forward(train)
train_res = result(g5, tra_output, train, idx2label)
Final_evalu(train_res)

,精确率,召回率,F1,support
cls,,,,
sms_other,NaN,0.000000,NaN,35
交易流水＿余额,0.894737,0.962264,0.927273,53
交易流水＿转账,0.871429,1.000000,0.931298,610
信用卡＿申请失败,NaN,0.000000,NaN,2
信用卡＿还款提醒,NaN,0.000000,NaN,45
信用卡＿逾期警告,NaN,0.000000,NaN,13
账号异常＿余额不足,0.835821,1.000000,0.910569,56
账号异常＿信用额度不足,NaN,0.000000,NaN,1
账号异常＿卡号冻结,NaN,0.000000,NaN,19


In [10]:
val_output = graphsage.forward(val)
val_res = result(g5, val_output, val, idx2label)
Final_evalu(val_res)

,精确率,召回率,F1,support
cls,,,,
sms_other,NaN,0.0,NaN,1
交易流水＿余额,NaN,0.0,NaN,2
交易流水＿转账,0.794872,1.0,0.885714,31
信用卡＿还款提醒,NaN,0.0,NaN,3
信用卡＿逾期警告,NaN,0.0,NaN,2
账号异常＿余额不足,0.800000,1.0,0.888889,4
账号异常＿扣款失败,0.500000,1.0,0.666667,3
账户账号＿自己,NaN,0.0,NaN,1
贷前申请＿申请交互,NaN,0.0,NaN,1
